In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
import os
os.getcwd()
os.chdir("/content/drive/My Drive/Colab Notebooks/mecab")

In [4]:
spam_data = pd.read_csv('../spam.csv', encoding='latin1')
# 3, 4, 5열 삭제
spam_data = spam_data.dropna(axis=1)

# 칼럼명 변경
spam_data.columns = ['label', 'mail']

# ham, spam 숫자로 변경
spam_data['label'] = spam_data['label'].replace('spam', 1)
spam_data['label'] = spam_data['label'].replace('ham', 0)

# 단어 아니면 삭제
spam_data['mail'] = spam_data['mail'].replace(r"[^\w]", " ")

# 공백이 있을 경우
spam_data['mail'] = spam_data['mail'].replace('', np.nan)
spam_data['label'] = spam_data['label'].replace('', np.nan)

# 결측치 제거
spam_data = spam_data.dropna(how='any', axis=0)

print("# preproessing done")


# split train/test
mail_train, mail_test, y_train, y_test = train_test_split(spam_data['mail'], spam_data
                                                          ['label'], test_size=0.2, shuffle=False)

print("# split done")

# 토큰화

stopwords = ['a', 'an']

X_train = []
for stc in mail_train:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

    
X_test = []
for stc in mail_test:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)

print("# tokenization done")

# preproessing done
# split done
# tokenization done


In [8]:
# X_train 정수 인덱스
# 빈도수가 2개 이상인 단어
tokenizer = Tokenizer(4000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print("# int_encoding done")


# int_encoding done


In [7]:
# print(len(tokenizer.word_index))

# low_count = 0
# for word, word_count in tokenizer.word_counts.items():
#     if word_count > 1:
#         low_count += 1
# print(low_count)

11817
4657


In [9]:
# padding
# max_len = max([len(x) for x in X_train]) # 153
# mean_len = np.mean([len(x) for x in X_train]) # 14
# length 차이가 커서 50으로 잡음. 효율성을 위해 너무 큰 값(outlier)을 덜 고려함
len = 50
X_train = pad_sequences(X_train, maxlen=len)
X_test = pad_sequences(X_test, maxlen=len)
print("# padding done")

# padding done


In [13]:
# 모델 생성 SimpleRNN
model = Sequential()
model.add(Embedding(4000, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

In [14]:
# 모델 훈련
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6)

Epoch 1/6
140/140 [==============================] - 2s 12ms/step - loss: 0.2855 - accuracy: 0.8966 - val_loss: 0.1341 - val_accuracy: 0.9641
Epoch 2/6
140/140 [==============================] - 1s 11ms/step - loss: 0.0965 - accuracy: 0.9726 - val_loss: 0.0779 - val_accuracy: 0.9776
Epoch 3/6
140/140 [==============================] - 2s 11ms/step - loss: 0.0565 - accuracy: 0.9829 - val_loss: 0.0639 - val_accuracy: 0.9839
Epoch 4/6
140/140 [==============================] - 2s 11ms/step - loss: 0.0525 - accuracy: 0.9816 - val_loss: 0.0596 - val_accuracy: 0.9839
Epoch 5/6
140/140 [==============================] - 2s 11ms/step - loss: 0.0291 - accuracy: 0.9921 - val_loss: 0.0637 - val_accuracy: 0.9848
Epoch 6/6
140/140 [==============================] - 2s 11ms/step - loss: 0.0219 - accuracy: 0.9928 - val_loss: 0.0633 - val_accuracy: 0.9812


In [15]:
# 모델 생성 LSTM
model = Sequential()
model.add(Embedding(4000, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

# callback
# early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
# model_check = ModelCheckpoint('LSTM_spam_best_sol.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [16]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6)

Epoch 1/6
140/140 [==============================] - 3s 25ms/step - loss: 0.2554 - accuracy: 0.9071 - val_loss: 0.1205 - val_accuracy: 0.9659
Epoch 2/6
140/140 [==============================] - 3s 23ms/step - loss: 0.0860 - accuracy: 0.9809 - val_loss: 0.0687 - val_accuracy: 0.9821
Epoch 3/6
140/140 [==============================] - 3s 23ms/step - loss: 0.0479 - accuracy: 0.9868 - val_loss: 0.0598 - val_accuracy: 0.9830
Epoch 4/6
140/140 [==============================] - 3s 24ms/step - loss: 0.0360 - accuracy: 0.9906 - val_loss: 0.0600 - val_accuracy: 0.9839
Epoch 5/6
140/140 [==============================] - 3s 24ms/step - loss: 0.0350 - accuracy: 0.9904 - val_loss: 0.0565 - val_accuracy: 0.9839
Epoch 6/6
140/140 [==============================] - 3s 24ms/step - loss: 0.0265 - accuracy: 0.9928 - val_loss: 0.0550 - val_accuracy: 0.9848


In [18]:
sentence = input()
# 토큰화
token_stc = sentence.split()
# 정수 인코딩
encode_stc = tokenizer.texts_to_sequences([token_stc])
# padding
pad_stc = pad_sequences(encode_stc, maxlen=len)
score = model.predict(pad_stc)

print(score)

URGENT! Your Mobile No 07808726822 was awarded a L2,000 Bonus Caller Prize on 02/09/03! This is our 2nd attempt to contact YOU! Call 0871-872-9758 BOX95QU
[[0.9974748]]
